<a href="https://colab.research.google.com/github/tnc-br/ddf-isoscapes/blob/npr-working/Data_Collection_for_Paper_Brazil_OOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab loads all the data in the DDF harness bigquery table and stores it into a dataframe.

First, update your bigquery api (you will need to restart the runtime after this step).

In [1]:
%pip install --upgrade google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0


### Dependencies

In [2]:
# This stub (ddfimport) allows the Ddf EE API to be imported.
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_import_common()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 18 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 7.36 KiB | 7.36 MiB/s, done.
Resolving deltas: 100% (7/7), done.
executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [3]:
import importlib
import eeddf
import bqddf
importlib.reload(eeddf)
importlib.reload(bqddf)

# Use your credentials to access the BQ tables.
eeddf.initialize_ddf(test_environment = True)

### Load the data

In [28]:
def get_all_training_results():
  client = bqddf._get_big_query_client()

  # Set up SQL query
  table_name = f"{bqddf._CONFIG['DATASET']}.{bqddf._CONFIG['FLATTENED_TABLE']}"
  query = f"SELECT * FROM {table_name}"

  # Execute the query
  results = client.query_and_wait(query)
  return results

results = get_all_training_results()

# Results is iterator based, so you will get an error running this twice!
df = results.to_dataframe()

import json
from typing import Set, Dict, Any
import numpy as np
import pandas as pd

def parse_tags(tags: np.ndarray) -> Set[str]:
  return set(tags.tolist())
df['tags'] = df['tags'].apply(parse_tags)

def find_by_tags(df: pd.DataFrame, tags: Set[str]) -> Dict[str, Any]:
  return df[df['tags'].apply(lambda x: all([tag in x for tag in tags]))].squeeze().to_dict()

def find_many_by_tags(df: pd.DataFrame, tags: Set[str]) -> Dict[str, Any]:
  return df[df['tags'].apply(lambda x: all([tag in x for tag in tags]))]

def print_experiment(experiment: Dict[str, Any]) -> None:
  print(f"RMSE (Means): {experiment['mean_rmse']:.3f}")
  print(f"RMSE (Variances): {experiment['var_rmse']:.3f}")

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/_pandas_helpers.py:208: UserWarning: Unable to determine Arrow type for field 'as_json'.
  warnings.warn(


In [ ]:
print_experiment(find_by_tags(df, {'author:npr', 'ordinary_kriging', 'all_standardized', 'linear_variogram'}))

RMSE (Means): 1.273
RMSE (Variances): 1.148


In [7]:
bqddf.add_tag(find_by_tags(df, {'author:npr', 'ordinary_kriging', 'all_standardized', 'linear_variogram'})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id 'test-okrige-2024-10-25-1' successfully.


[Row((['author:npr', 'ordinary_kriging', 'all_standardized', 'linear_variogram', 'main_ood_final_result'],), {'tags': 0})]

In [ ]:
print_experiment(find_by_tags(df, {'author:npr', 'ordinary_kriging', 'all_standardized', 'gaussian_variogram', 'rev2'}))

RMSE (Means): 1.269
RMSE (Variances): 1.190


In [8]:
bqddf.add_tag(find_by_tags(df, {'author:npr', 'ordinary_kriging', 'all_standardized', 'gaussian_variogram', 'rev2'})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id 'test-okrige-2025-01-17-4' successfully.


[Row((['author:npr', 'ordinary_kriging', 'all_standardized', 'gaussian_variogram', 'rev2', 'main_ood_final_result'],), {'tags': 0})]

In [ ]:
print_experiment(find_by_tags(df, {'author:npr', 'universal_kriging', 'all_standardized', 'linear_variogram', 'rev2'}))

RMSE (Means): 1.183
RMSE (Variances): 1.157


In [9]:
bqddf.add_tag(find_by_tags(df, {'author:npr', 'universal_kriging', 'all_standardized', 'linear_variogram', 'rev2'})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id 'test-ukrige-2025-01-17-2' successfully.


[Row((['author:npr', 'universal_kriging', 'all_standardized', 'linear_variogram', 'rev2', 'main_ood_final_result'],), {'tags': 0})]

In [ ]:
print_experiment(find_by_tags(df, {'author:npr', 'universal_kriging', 'all_standardized', 'gaussian_variogram', 'rev2'}))

RMSE (Means): 1.246
RMSE (Variances): 1.247


In [10]:
bqddf.add_tag(find_by_tags(df, {'author:npr', 'universal_kriging', 'all_standardized', 'gaussian_variogram', 'rev2'})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id 'test-ukrige-2025-01-17-3' successfully.


[Row((['author:npr', 'universal_kriging', 'all_standardized', 'gaussian_variogram', 'rev2', 'main_ood_final_result'],), {'tags': 0})]

In [ ]:
print_experiment(find_by_tags(df, {'author:npr', "regression_kriging", "all_standardized", "rev1", "krige_type:universal", "variogram:gaussian", "n_estimators:100", "max_depth:20", "regression_strategy:gradientboosting"}))

RMSE (Means): 0.639
RMSE (Variances): 1.253


In [11]:
bqddf.add_tag(find_by_tags(df, {'author:npr', "regression_kriging", "all_standardized", "rev1", "krige_type:universal", "variogram:gaussian", "n_estimators:100", "max_depth:20", "regression_strategy:gradientboosting"})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id 'test-rkrige-2025-01-17-1' successfully.


[Row((['author:npr', 'regression_kriging', 'all_standardized', 'rev1', 'krige_type:universal', 'variogram:gaussian', 'n_estimators:100', 'max_depth:20', 'regression_strategy:gradientboosting', 'main_ood_final_result'],), {'tags': 0})]

In [ ]:
print_experiment(find_by_tags(df, {'author:npr', "regression_kriging", "all_standardized", "rev1", "krige_type:universal", "variogram:linear", "n_estimators:100", "max_depth:20", "regression_strategy:gradientboosting"}))

RMSE (Means): 0.659
RMSE (Variances): 1.253


In [13]:
bqddf.add_tag(find_by_tags(df, {'author:npr', "regression_kriging", "all_standardized", "rev1", "krige_type:universal", "variogram:linear", "n_estimators:100", "max_depth:20", "regression_strategy:gradientboosting"})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id 'test-rkrige-2025-01-17-2' successfully.


[Row((['author:npr', 'regression_kriging', 'all_standardized', 'rev1', 'krige_type:universal', 'variogram:linear', 'n_estimators:100', 'max_depth:20', 'regression_strategy:gradientboosting', 'main_ood_final_result'],), {'tags': 0})]

In [38]:
print_experiment(find_by_tags(df, {'author:npr', "ViNN", "rev2"}))

RMSE (Means): 0.848
RMSE (Variances): 1.683


In [39]:
bqddf.add_tag(find_by_tags(df, {'author:npr', "ViNN", "rev2"})['training_id'], 'main_ood_final_result')

Tag 'main_ood_final_result' added for training_id '2025-01-31-vinn-1' successfully.


[Row((['author:npr', 'ViNN', 'rev2', 'main_ood_final_result'],), {'tags': 0})]